In [1]:
from __future__ import print_function
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
import gensim 
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from pprint import pprint
import nltk
import spacy
import collections

# Replace with the path of the dataset (used excel instead as I was having issues with the decoding of the CSV)
DATASET_PATH = \
'C:/Users/yogas/OneDrive/NUS/BT4014 Analytics Driven Design of Adaptive Systems/Assignments/A3/data.xlsx'

C:\Users\yogas\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Data Pre-processing

In [2]:
dataset = pd.read_excel(DATASET_PATH)
print(dataset.shape) # check the shape of the original dataset

dataset = dataset.dropna() # delete rows with missing values
dataset.reviewTxt = dataset.reviewTxt.astype(str)
print(dataset.head())
print(dataset.shape) # check whether too many rows have been omitted from dropping missing values

# group the reviews by their products
product_texts = dataset.groupby('productID_index')['reviewTxt'].agg(lambda col: ' '.join(col))
review_text = product_texts.values.tolist()

(327028, 6)
                                title  \
0             Nike Flight headphones.   
1       Good outdoor usage headphones   
2  Sound very much above their price.   
3                     Not good at all   
4                          Very nice.   

                                           reviewTxt   reviewTime  \
0  the unit right out of the box is faulty, does ...  01 04, 2013   
1  i go through a lot of headphones and these are...  27 12, 2012   
2  listening to them right now. very impressed. i...  16 01, 2016   
3  not good at all, no extra bass. the difference...  16 05, 2015   
4  the more i listen the more i listen like them....  19 03, 2015   

   productID_index  reviewID_index  reviewerID_index  
0                1               1                 1  
1                1               2                 2  
2                2               3                 3  
3                2               4                 4  
4                2               5                 

## Tokenize words and clean up text

In [3]:
def set_to_words(reviews):
    for review in reviews:
        yield(gensim.utils.simple_preprocess(str(review), deacc=True))  # deacc=True to remove punctuations

review_words = list(set_to_words(review_text))

## Stopwords

In [4]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

review_words_nostops = remove_stopwords(review_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yogas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Create Bigram and Trigram models

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(review_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[review_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

review_words_bigrams = make_bigrams(review_words_nostops)

C:\Users\yogas\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## Lemmatization

In [6]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

In [7]:
nlp.max_length = 10000000 # set the number of characters limit to above the default to prevent value error
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

review_lemmatized = lemmatization(review_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(review_lemmatized)

# Create Corpus
texts = review_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abuse', 1),
  ('amazon', 1),
  ('arrangement', 1),
  ('backboards', 1),
  ('bag', 1),
  ('bench', 1),
  ('box', 1),
  ('buying', 1),
  ('chance', 1),
  ('check', 1),
  ('contact', 1),
  ('faulty', 1),
  ('favorit', 1),
  ('fifth', 1),
  ('fourth', 1),
  ('get', 2),
  ('go', 1),
  ('good', 2),
  ('head', 1),
  ('headphone', 3),
  ('history', 1),
  ('last', 1),
  ('lot', 1),
  ('modality', 1),
  ('must', 1),
  ('play', 1),
  ('pretty', 1),
  ('purchase', 2),
  ('rain', 1),
  ('regard', 1),
  ('return', 1),
  ('right', 1),
  ('sound', 2),
  ('stuff', 1),
  ('sturdiness', 1),
  ('sweat', 1),
  ('take', 1),
  ('tired', 1),
  ('unit', 1),
  ('use', 1),
  ('useless', 1),
  ('want', 1),
  ('way', 1),
  ('weightlifting', 1),
  ('work', 1),
  ('year', 1)]]

# LDA Model

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the keywords in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"comply_tsx" + 0.002*"ckx" + 0.001*"champagne" + 0.001*"tim_mcgraw" + '
  '0.001*"sonicfuel" + 0.001*"ellie" + 0.001*"ath_ckx" + 0.001*"composure" + '
  '0.001*"stagger" + 0.001*"quotation"'),
 (1,
  '0.041*"great" + 0.035*"love" + 0.029*"gift" + 0.026*"sound" + 0.026*"cute" '
  '+ 0.018*"good" + 0.016*"bud" + 0.016*"earbud" + 0.015*"ear" + 0.014*"work"'),
 (2,
  '0.051*"cord" + 0.026*"work" + 0.023*"use" + 0.017*"extension" + '
  '0.012*"short" + 0.012*"earbud" + 0.011*"cable" + 0.011*"pull" + 0.011*"get" '
  '+ 0.009*"retractable"'),
 (3,
  '0.027*"sound" + 0.021*"headphone" + 0.019*"work" + 0.017*"good" + '
  '0.016*"quality" + 0.014*"use" + 0.014*"great" + 0.013*"mic" + 0.013*"phone" '
  '+ 0.011*"control"'),
 (4,
  '0.029*"sound" + 0.024*"earbud" + 0.021*"good" + 0.019*"bud" + 0.019*"ear" + '
  '0.018*"buy" + 0.016*"work" + 0.016*"pair" + 0.015*"get" + 0.015*"great"'),
 (5,
  '0.001*"ashtray" + 0.000*"margaritaville" + 0.000*"comfortbaly" + '
  '0.000*"ear" + 0.000*"

## Compute Model Perplexity and Coherence Score

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=review_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.612156377259604

Coherence Score:  0.3221151774225745


# Topic distribution over words

In [13]:
flat_list_texts = []
for sublist in texts:
    for item in sublist:
        flat_list_texts.append(item)
        

counter = collections.Counter(flat_list_texts)
top_10_words = counter.most_common(10)
print(top_10_words)

[('sound', 226426), ('ear', 203750), ('good', 177273), ('headphone', 154136), ('great', 135627), ('use', 113918), ('quality', 106016), ('work', 101765), ('get', 91841), ('earbud', 89029)]


In [14]:
for word in top_10_words:
    print('{}: {}\n'.format(word[0], lda_model.get_term_topics(word[0])))

sound: [(1, 0.026115233), (3, 0.026521266), (4, 0.028721593), (6, 0.023475686), (7, 0.021024453), (8, 0.036127236), (9, 0.035294153), (10, 0.0289878), (11, 0.013684696), (14, 0.028221186), (17, 0.020802133), (18, 0.01605638)]

ear: [(1, 0.014523878), (4, 0.018561978), (6, 0.01208895), (7, 0.021116342), (8, 0.022628812), (9, 0.039128363), (10, 0.013558157), (11, 0.029688701), (14, 0.026823092), (16, 0.010903125), (17, 0.053848572), (18, 0.01517913)]

good: [(1, 0.017766505), (3, 0.016850708), (4, 0.020849999), (6, 0.016027603), (7, 0.010638317), (8, 0.0313152), (9, 0.028879112), (10, 0.013650272), (11, 0.013133186), (14, 0.02144401), (17, 0.014128913), (18, 0.015831769), (19, 0.010877824)]

headphone: [(3, 0.020591648), (6, 0.03376935), (7, 0.02503506), (8, 0.026653089), (9, 0.014836319), (10, 0.010003738), (14, 0.018167006), (17, 0.025189938), (18, 0.013287235)]

great: [(1, 0.04076711), (3, 0.01375215), (4, 0.015222011), (6, 0.013648306), (8, 0.025822936), (9, 0.016037587), (14, 0.025

# Product distribution over topics

In [15]:
products = dataset.productID_index.unique()
    
# product distribution for each topic
product_distribution = list()
for i in range(20): # 20 topics
    product_distribution.append(list())

for product in products:
    topic_distribution = lda_model.get_document_topics(id2word.doc2bow(texts[product - 1]))
    for topic in topic_distribution:
        # derive the product's importance to the topic from the topic's importance to the product
        # assume all products contributes the same amount to the corpus
        product_distribution[topic[0]].append([product, topic[1]]) 

# standardize the importance of each product (so the probability sums to 1)
for topic, distribution in enumerate(product_distribution):
    total_weight = sum(x[1] for x in distribution)
    for product in distribution:
        product[1] /= total_weight
    print('Topic {} distribution: {} \n'.format(topic, distribution))

Topic 0 distribution: [[24, 0.2576710395901723], [42, 0.19654210608071923], [80, 0.12252967764209408], [84, 0.1643205096119434], [92, 0.258936667075071]] 

Topic 1 distribution: [[13, 0.0036065980548983098], [17, 0.010493825578316716], [19, 0.009635513653113067], [32, 0.002214902811015383], [43, 0.0009725966087703779], [44, 0.011496103501942618], [76, 0.006678215347644906], [79, 0.0003683287455427124], [82, 0.0028137792742201915], [87, 0.00046641989782379814], [88, 0.0009684157448445444], [91, 0.0033862728999308755], [100, 0.0036117864063834057], [112, 0.004681132501720415], [116, 0.0003683287455427124], [119, 0.03016066563936811], [140, 0.00281673097841674], [158, 0.004837356812645949], [159, 0.0009751884070279561], [167, 0.016011967287058847], [171, 0.0022979021698670953], [176, 0.002070895376426514], [177, 0.0005227742448567021], [184, 0.03016071273621322], [187, 0.005971225132932329], [188, 0.005971716026971767], [190, 0.010191427604354293], [192, 0.00408721728357932], [196, 0.0037


Topic 4 distribution: [[4, 0.0001583062641879787], [6, 0.00019073993101097024], [7, 0.005170075459305607], [8, 0.0015998976126781765], [11, 0.0007307632761219537], [13, 0.0022634266173037583], [15, 0.007373942873779746], [16, 0.005582428793745709], [17, 0.005633261835753849], [18, 0.0007843209912728167], [19, 0.00017672998257508137], [22, 0.001017736965865957], [24, 0.0008095130704446305], [25, 0.000653516083198643], [26, 0.004121893533805701], [27, 0.0028034027019724515], [28, 0.00965560894354116], [29, 0.0011869586833324552], [30, 0.000150945371081192], [33, 0.0007823717243714608], [35, 0.003509141438095883], [36, 0.0032249598717426996], [37, 0.006342158692084111], [40, 0.00023539443975625603], [43, 0.001863642025469964], [45, 0.004731393531647581], [46, 0.00013772680871054936], [47, 0.0004004541147662387], [48, 0.00018914549089292962], [50, 0.0003680749400480429], [51, 0.0001267098631795256], [56, 0.0005334488472037701], [59, 0.00021951343728219804], [60, 0.00229596697095373], [61,


Topic 5 distribution: [[176, 1.0]] 

Topic 6 distribution: [[1, 0.004986057711735299], [3, 0.0013450538500046519], [4, 0.0009962273327537284], [6, 0.003329519404765803], [7, 0.0002485911647154283], [8, 0.0005280897869092724], [9, 0.0003938734998393853], [10, 0.0004283583691606741], [11, 0.0006474937130861072], [12, 0.00014438974282076624], [13, 0.00023102519651258641], [15, 0.001897959015802864], [17, 0.00019816442689972685], [18, 0.00045116684272129745], [19, 0.005612585718527804], [22, 0.002164574428538849], [23, 0.0014108578246406194], [25, 0.000477119138221309], [26, 0.0003757075443308903], [27, 0.0005046010685559317], [28, 0.00016515148762587524], [29, 0.000682942974935192], [30, 0.00012829547058950525], [31, 0.0031011232464986304], [32, 0.003692687240869292], [33, 0.0007942818276617589], [34, 0.0014827331977730228], [35, 0.0004676560416309156], [36, 0.00044043069914359974], [37, 0.002370024802312673], [38, 0.007006665637133135], [42, 0.0005123701072885072], [43, 0.00677067939621


Topic 7 distribution: [[1, 0.008445113041419051], [4, 0.005842492109740257], [18, 0.0015275387672022533], [26, 0.0013621881578444561], [35, 0.002137615064117669], [39, 0.006586718978447712], [41, 0.011734363557764709], [47, 0.0019244176833975927], [48, 0.001446724409701565], [49, 0.0028055068423207875], [51, 0.008963989489649826], [53, 0.008852903140091623], [54, 0.003614987511953448], [56, 0.0012431189384749944], [57, 0.01524337509959145], [61, 0.005866730297283749], [67, 0.0022363158439229944], [73, 0.010980644260158262], [77, 0.0047952697922138965], [79, 0.0018177956080257637], [81, 0.008872465833987405], [84, 0.006616944483364852], [90, 0.003376819464772176], [91, 0.0023678614385548276], [93, 0.007276785028314985], [94, 0.0019020842427822174], [95, 0.014939787859103194], [106, 0.0037189693440585806], [113, 0.02890277623215501], [116, 0.0018178150326726545], [124, 0.0016759937998356437], [127, 0.0019383291251698158], [137, 0.0016120233457268568], [153, 0.0013846145727405802], [158,


Topic 9 distribution: [[1, 0.00011503154579882885], [3, 0.00024847596363757816], [4, 0.0017161122346820623], [5, 0.0009139477048148436], [6, 0.006035936494529845], [7, 0.0009448241233542625], [8, 0.0023413077777175864], [9, 0.001037059111073348], [11, 0.0001288017233719712], [12, 0.0017457610970502636], [13, 0.0009676148433971326], [15, 0.00024254870064844468], [16, 0.00011273457454775796], [17, 0.0007439433739842113], [18, 0.0009512431058909127], [19, 0.0002743981020974523], [20, 0.0048931300764108255], [21, 0.00015906191533568965], [22, 0.0026108428648074513], [23, 0.0017545678540950846], [24, 0.0029800694545807863], [26, 0.0010961563558808225], [27, 0.00033788020055212437], [28, 0.001112875308451712], [29, 0.0021840772806895267], [30, 0.003182506781424051], [31, 0.00010617861598178104], [32, 0.00017125159691141345], [33, 0.0005382479694642545], [34, 0.002617238704320317], [35, 0.0004947458812700157], [36, 0.0007931889402814157], [38, 0.0002409688407356407], [39, 0.00514451127374726

...


Topic 11 distribution: [[2, 0.0015739863128624325], [5, 0.0011293102050165375], [9, 0.004475224549515695], [11, 0.000758602003987448], [12, 0.000804028522768945], [18, 0.0008522049894021268], [21, 0.037864132433667534], [25, 0.0018670650459118327], [34, 0.0021302173706180035], [35, 0.0005216753762474923], [36, 0.0013062328985441896], [41, 0.002508688664155947], [42, 0.0024774749384877588], [45, 0.001569602409159111], [50, 0.0007432786638597981], [51, 0.0030037697715878404], [54, 0.0029054158444192474], [55, 0.0005191921560054947], [60, 0.0035989426002180887], [61, 0.0007882239532707412], [62, 0.0017210329833371267], [65, 0.0021601275206940525], [67, 0.0005045296530867792], [70, 0.0007104982394169412], [72, 0.0018214567719736484], [77, 0.004736869150010696], [78, 0.0020646634238268473], [79, 0.0013395396460420443], [81, 0.0010547649763349464], [86, 0.0007237296313740781], [87, 0.00354679313457759], [93, 0.005317839308599654], [94, 0.002146126545193727], [95, 0.0013693734129383208], [96


Topic 12 distribution: [[16, 0.03668527030361218], [38, 0.059120911338220894], [56, 0.04327008918923769], [68, 0.13199146780634266], [69, 0.13203593165490196], [75, 0.04366672219877868], [82, 0.05917477399619626], [114, 0.01857722813395771], [154, 0.039429859244796105], [169, 0.02538192224671326], [180, 0.09808955897189305], [185, 0.04366428631357016], [404, 0.13204927861470941], [591, 0.03642094911493038], [686, 0.03981146391582096], [773, 0.02313110902692422], [793, 0.03749917792939441]] 

Topic 13 distribution: [[31, 0.18156914192889712], [118, 0.06982109919950039], [145, 0.023216493513784887], [175, 0.04403444239840725], [192, 0.04368721688974912], [197, 0.10513809936578072], [268, 0.07826190371804691], [383, 0.051734113185164174], [414, 0.025649900721895318], [421, 0.03493555805349426], [461, 0.0771282138967552], [487, 0.22454770036755686], [796, 0.040276116760967824]] 

Topic 14 distribution: [[3, 0.01535358604591603], [5, 0.00375874308372677], [7, 0.009750102967928673], [8, 0.0


Topic 15 distribution: [[420, 0.05304073108511111], [431, 0.08555689940557697], [483, 0.08120345875152778], [547, 0.05712568675585432], [561, 0.0991112853273967], [567, 0.09465095225325253], [711, 0.09911132831938761], [726, 0.13488285735777616], [814, 0.29531680074411687]] 

Topic 16 distribution: [[36, 0.005160815244190913], [45, 0.009387765776133343], [50, 0.019941301534253154], [54, 0.006886302824355862], [63, 0.008129106966305883], [103, 0.06967183469867508], [150, 0.10798999198842923], [164, 0.030926955760391856], [201, 0.030935135584685785], [224, 0.14369286367427703], [228, 0.004994027452865432], [238, 0.06689955447636584], [249, 0.009381691332955168], [273, 0.005407681322788149], [276, 0.14380057908106195], [288, 0.01696062224686486], [308, 0.004991097269922656], [344, 0.007503984870764104], [370, 0.010700671599268467], [392, 0.028695027341820174], [394, 0.01200313424866911], [412, 0.03192924492729493], [433, 0.014587478778072347], [440, 0.013952440069878747], [532, 0.0092767


Topic 18 distribution: [[1, 0.005553077185459313], [5, 0.000655565366055131], [8, 0.0007813105121521093], [20, 0.004676657679109577], [25, 0.00020549792092166544], [34, 0.0006436361244736864], [35, 0.0003394462540869039], [39, 0.0020014125787281315], [42, 0.00205351024878918], [47, 0.0017177326059920174], [48, 0.003442753658704882], [49, 0.010372020851773819], [50, 0.001295054489828447], [52, 0.00762480223475237], [53, 0.005710056005269197], [55, 0.00903260742048863], [56, 0.0006555538563561149], [57, 0.0058026433928470185], [58, 0.008812520078800427], [59, 0.011160529557763027], [61, 0.00039904021217145914], [63, 0.0006040822016317952], [64, 0.010494585110076863], [67, 0.011827774320713644], [68, 0.0003629971686160247], [69, 0.0003699858368042669], [70, 0.01584833227493694], [72, 0.00020387894841097844], [79, 0.0006463984522375479], [80, 0.00558879767671793], [81, 0.00046403814939679746], [85, 0.012984852784532137], [86, 0.0003090213121826067], [93, 0.0001930794118603997], [94, 0.000


Topic 19 distribution: [[5, 0.0018847530158230123], [11, 0.045699468278490384], [13, 0.002697472313166062], [23, 0.0007116673614856088], [25, 0.001170253767051798], [27, 0.0008215721832526574], [31, 0.018692171827785063], [46, 0.011633376514501973], [56, 0.0036728240588744738], [61, 0.0013661111653904023], [72, 0.0011658603448077562], [73, 0.0026998200078746657], [75, 0.0013604353832880052], [79, 0.0009323374891642997], [87, 0.0017871612502024135], [91, 0.0008606132255538253], [109, 0.0019347116834936037], [116, 0.0009323363217654343], [123, 0.031213280243482508], [142, 0.013629321715088384], [146, 0.0044758277069560145], [148, 0.0063668889015239], [149, 0.006736420229057859], [154, 0.0017791476684865571], [158, 0.007407774639383311], [161, 0.0017079146574366987], [163, 0.006156477374759942], [177, 0.0013578619914654374], [182, 0.0067328459872747475], [185, 0.0013608307423703975], [192, 0.004152647228694551], [211, 0.0022692450601299935], [212, 0.0013962252753396472], [217, 0.00093233

# Peers for each product

In [16]:
# store the topic distribution of each product
topic_distribution = list()
for product in products:
    topic_distribution.append(id2word.doc2bow(texts[product - 1]))

In [17]:
# get the 5 most similar peers, by their topic distribution
peers = list()
for product_id in products:
    peer_similarity_scores = list()
    for peer_id in products:
        similarity_score = gensim.matutils.cossim(topic_distribution[product_id - 1], topic_distribution[peer_id - 1])
        peer_similarity_scores.append([peer_id, similarity_score])
    peer_similarity_scores.sort(reverse=True, key=lambda x: x[1])
    # skip the most similar score as it will be most similar to itself
    peers.append([product_id, peer_similarity_scores[1][0], peer_similarity_scores[1][1],
                 peer_similarity_scores[2][0], peer_similarity_scores[2][1], 
                 peer_similarity_scores[3][0], peer_similarity_scores[3][1],
                 peer_similarity_scores[4][0], peer_similarity_scores[4][1],
                 peer_similarity_scores[5][0], peer_similarity_scores[5][1]])

peers_df = pd.DataFrame(peers, columns=['product_id', '1st_peer_id', '1st_similarity', '2nd_peer_id', '2nd_similarity', 
                                        '3rd_peer_id', '3rd_similarity',  '4th_peer_id', '4th_similarity', 
                                        '5th_peer_id', '5th_similarity'])
peers_df.to_csv('peers.csv', index=False)